In [1]:
import pandas as pd

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
tp_df = pd.read_csv("dataset/train_true_positive_features.csv")
tn_df = pd.read_csv("dataset/train_true_negative_features.csv")
df = pd.concat([tp_df, tn_df])
df = df.reset_index(drop = True)

In [4]:
df = df[['vgg_cosine', 'vgg_euclidean_l2'
         , 'facenet_cosine', 'facenet_euclidean_l2'
         , 'openface_cosine', 'openface_euclidean_l2'
         , 'is_related']]

In [5]:
df.sample(5)

,vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
284475,0.630001,1.122498,0.910058,1.349117,0.622130,1.115464,0
133369,0.437769,0.935702,0.398903,0.893199,0.270992,0.736195,1
171553,0.615905,1.109870,0.925502,1.360516,1.252341,1.582619,0
335908,0.652894,1.142711,0.984363,1.403113,0.781777,1.250422,0
350565,0.740999,1.217374,1.210697,1.556083,0.803363,1.267567,0


# Train Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test = train_test_split(df, test_size=0.15, random_state=17)

# H2O AutoML

In [8]:
import h2o
from h2o.automl import H2OAutoML

In [9]:
h2o.__version__

'3.26.0.3'

In [10]:
# specify maximum memory size and number of threads based on your system

In [11]:
import multiprocessing
multiprocessing.cpu_count()

72

In [12]:
import psutil
psutil.virtual_memory()

svmem(total=1622764929024, available=680764293120, percent=58.0, used=938262872064, free=587229573120, active=921854357504, inactive=94388834304, buffers=0, cached=97272483840, shared=855113728, slab=2700742656)

In [13]:
#h2o.init()
h2o.init(ip="127.0.0.1", max_mem_size_GB = 200, nthreads = 10)

Checking whether there is an H2O instance running at http://127.0.0.1:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_212"; OpenJDK Runtime Environment (build 1.8.0_212-b04); OpenJDK 64-Bit Server VM (build 25.212-b04, mixed mode)
  Starting server from /workspaces/96273/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpy29fyfrx
  JVM stdout: /tmp/tmpy29fyfrx/h2o_96273_started_from_python.out
  JVM stderr: /tmp/tmpy29fyfrx/h2o_96273_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Istanbul
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_96273_4w02yc
H2O cluster total nodes:,1
H2O cluster free memory:,177.8 Gb
H2O cluster total cores:,72
H2O cluster allowed cores:,10
H2O cluster status:,"accepting new members, healthy"


In [14]:
hf = h2o.H2OFrame(x_train)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
hf.head()

vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
0.461026,0.960236,0.759346,1.23235,0.380667,0.872545,1
0.523298,1.02303,0.820089,1.28069,0.572227,1.06979,1
0.57931,1.07639,0.834568,1.29195,0.411612,0.907317,1
0.661336,1.15007,1.25314,1.58312,0.809752,1.2726,0
0.380073,0.871864,0.419412,0.915873,0.386296,0.878972,1
0.586928,1.08345,0.980849,1.40061,0.791823,1.25843,0
0.543978,1.04305,0.900194,1.34179,0.88473,1.33021,1
0.378311,0.86984,0.799367,1.26441,0.467552,0.967008,0
0.483638,0.983502,0.590177,1.08644,0.411636,0.907343,1
0.874395,1.32242,0.935428,1.36779,1.11803,1.49535,0


In [16]:
y_label = "is_related"
x_labels = list(df.drop(columns=[y_label]).columns)

In [17]:
x_labels

['vgg_cosine',
 'vgg_euclidean_l2',
 'facenet_cosine',
 'facenet_euclidean_l2',
 'openface_cosine',
 'openface_euclidean_l2']

In [18]:
aml = H2OAutoML(max_runtime_secs=60*60*2)

In [19]:
aml.train(x = x_labels, y = y_label, training_frame = hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%


# Leaderboard

In [20]:
lb = aml.leaderboard

In [21]:
lb.head()
#lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20190903_211714,0.198223,0.445223,0.198223,0.396509,0.312694
StackedEnsemble_BestOfFamily_AutoML_20190903_211714,0.198223,0.445223,0.198223,0.396509,0.312694
GBM_1_AutoML_20190903_211714,0.19834,0.445353,0.19834,0.397159,0.312776
XGBoost_3_AutoML_20190903_211714,0.198346,0.44536,0.198346,0.397811,0.312951
GBM_2_AutoML_20190903_211714,0.198461,0.445489,0.198461,0.397165,0.312894
GBM_3_AutoML_20190903_211714,0.19858,0.445623,0.19858,0.396989,0.312967
DeepLearning_1_AutoML_20190903_211714,0.199118,0.446226,0.199118,0.398667,0.31425
XGBoost_1_AutoML_20190903_211714,0.199229,0.446351,0.199229,0.397684,0.313674
GBM_4_AutoML_20190903_211714,0.199367,0.446506,0.199367,0.396856,0.313593
GLM_grid_1_AutoML_20190903_211714_model_1,0.199736,0.446919,0.199736,0.399971,0.314168


In [22]:
saved_model = h2o.save_model(aml.leader, path = "")

In [23]:
saved_model

'/outputs/USUALSUSPECTS/sefik/kinship/StackedEnsemble_AllModels_AutoML_20190903_211714'

# Validation Score

In [24]:
hf_val = h2o.H2OFrame(x_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [25]:
predictions = aml.predict(hf_val)
predictions_pd = predictions.as_data_frame()

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [26]:
actuals = hf_val['is_related']
actuals_pd = actuals.as_data_frame()

In [27]:
classified = 0
for i in range(0, predictions_pd.shape[0]):
    prediction = predictions_pd.iloc[i].values[0]
    
    if prediction > 0.5:
        prediction = 1
    else:
        prediction = 0
    
    actual = actuals_pd.iloc[i].values[0]
    
    if actual == prediction:
        classified = classified + 1

In [28]:
print("accuracy on test set is: ",100*classified / predictions_pd.shape[0],"%")

accuracy on test set is:  72.6397253351565 %
